In [1]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
# Computational modules
import zarr
import dask
import numpy as np
import pandas as pd
import xarray as xr
import gcsfs
import cartopy.crs as ccrs
import datetime
# Maps   
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import rioxarray
import geopandas as gpd
from shapely.geometry import box, mapping
import dateutil
import pysolar
from cartopy.util import add_cyclic_point

#!{sys.executable} -m pip install --user cmip6_preprocessing

# Local files and utility functions
sys.path.append("./subroutines/")
import pices
#!{sys.executable} -m pip install --user pytz
#!{sys.executable} -m pip install --user tzwhere
import pytz
#from tzwhere import tzwhere
#!{sys.executable} -m pip install --user astropy
#import astropy

import xesmf as xe
np.seterr(divide='ignore', invalid='ignore')
get_ipython().run_line_magic("matplotlib", "inline")
plt.rcParams["figure.figsize"] = 12, 6
get_ipython().run_line_magic("config", "InlineBackend.figure_format = 'retina'")

In [2]:
from dask.distributed import Client, progress
client = Client() 
client

Client Scheduler: tcp://127.0.0.1:41697 Dashboard: /user/trondkr/proxy/8787/status,Cluster Workers: 4 Cores: 4 Memory: 15.77 GB


In [3]:
class Config_pices():
    df = pd.read_csv(
    "https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv"
)
    fs = gcsfs.GCSFileSystem(token="anon", access="read_only")
   
    grid_labels = ["gn"]  # Can be gr=grid rotated, or gn=grid native
    member_ids = ["r1i1p1f1"]  #
    experiment_ids = ["ssp585"]  #'abrupt-4xCO2',
    source_ids = ["CanESM5"]
    variable_ids = ["uas","vas","tos"]
    table_ids = ["Amon","Amon","Omon"]  # Amon=atmospheric variables, Omon=Ocean variables, SImon=sea-ice variables
    dset_dict = {}
    start_date="1950-01-01"
    end_date="2100-08-01"
    clim_start="1961-01-01"
    clim_end="1990-01-01"
    
    selected_depth=0
  
# Create the object
config_pices_obj = Config_pices()
config_pices_obj = pices.get_and_organize_cmip6_data(config_pices_obj)

Running historical query on data: 
 ==> source_id=='CanESM5'and table_id=='Amon' and grid_label=='gn' and experiment_id=='historical' and variable_id=='uas'

Running projections query on data: 
 ==> source_id=='CanESM5'and table_id=='Amon' and member_id=='r1i1p1f1' and grid_label=='gn' and experiment_id=='ssp585' and variable_id=='uas'

CanESM5 => Dates extracted range from 1950-01-16 12:00:00 to 2100-07-16 12:00:00

Running historical query on data: 
 ==> source_id=='CanESM5'and table_id=='Amon' and grid_label=='gn' and experiment_id=='historical' and variable_id=='vas'

Running projections query on data: 
 ==> source_id=='CanESM5'and table_id=='Amon' and member_id=='r1i1p1f1' and grid_label=='gn' and experiment_id=='ssp585' and variable_id=='vas'

CanESM5 => Dates extracted range from 1950-01-16 12:00:00 to 2100-07-16 12:00:00

Running historical query on data: 
 ==> source_id=='CanESM5'and table_id=='Omon' and grid_label=='gn' and experiment_id=='historical' and variable_id=='tos'



In [4]:
# Following Roland Seferian, equation 3
# https://www.geosci-model-dev.net/11/321/2018/gmd-11-321-2018.pdf

def calculate_alpha_dir(n_lambda,µ):
    a = np.sqrt(1.0 - (1.0 - µ**2)/n_lambda**2)
    b = ((a-n_lambda*µ)/(a+n_lambda*µ))**2    
    c = ((µ-n_lambda*a)/(µ+n_lambda*a))**2

    return 0.5*(b+c)

In [5]:
def calculate_diffuse_reflection(n_λ,σ):
    # Diffuse albedo from Jin et al., 2006 (Eq 5b) 
    return -0.1479 + 0.1502*n_λ-0.0176*n_λ*σ

In [6]:
def surface_roughness(µ, σ):
    # Surface roughness following Jin et al. 2014 equation 4
    # This rougness parameter determines the Fresnel refraction 
    # index from flat surface
    return (0.0152-1.7873*µ + 6.8972*(µ**2)-8.5778*(µ**3)+ 4.071*σ-7.6446*µ*σ) * np.exp(0.1643-7.8409*µ-3.5639*µ**2-2.3588*σ+10.054*µ*σ)

In [7]:
def calculate_direct_reflection(n_λ,µ,σ):
    # Direct reflection following Jin et al. 2014 equation 1
    f_0 = calculate_alpha_dir(1.34,µ)
    f_λ = calculate_alpha_dir(n_λ,µ)
    
    return f_λ - (surface_roughness(µ, σ)*f_λ/f_0)

In [8]:
def calculate_direct_reflection_from_chl(λ, chl, alpha_chl, alpha_w, beta_w, σ, µ, alpha_direct):
   
    rw=0.48168549-0.014894708*σ-0.20703885*σ**2
 
    # Determine absorption and backscattering
    # coefficients to determine reflectance below the surface (Ro) once for all
    # Backscattering by chlorophyll:
    a_bp = 0.06*alpha_chl*np.exp(np.log(chl)*0.65) + 0.2*(0.00635+0.06*(np.exp(np.log(chl)*0.65))*np.exp(0.014*(440.0-λ)))
   
    # Backscattering of biological pigment (b_chl) with λ expressed here in nm and [Chl] in mg m−3. This
    # formulation is valid for [Chl] ranging between 0.02 and 2 mg m−3 (Morel and Maritorena (2001))
    # Equation 12 Roland Seferian, 2018
    b_chl=(0.416*np.exp(np.log(chl)*0.766))*(0.002+0.01*(0.5-0.25*np.log10(chl))*(np.exp(0.5*np.log10(chl)-0.3))*np.log(λ/550.0))
                                                                                 
   
    # # Use Morel 91 formula to compute the direct reflectance below the surface (Morel-Gentili(1991), Eq (12))
    n=0.5*beta_w/(0.5*beta_w + b_chl)
   
    # Equation 11 Roland Seferian, 2018
    beta = 0.6279-0.2227*n-0.0513*n**2 +(0.2465*n - 0.3119)*µ
    
    # Equation 10 Roland Seferian, 2018
    R0 = beta * (0.5*beta_w + b_chl)/(alpha_w + a_bp)
   
    # Water leaving albedo, equation 8 Roland Seferian, 2018
    return (R0*(1.0-rw)/(1-rw*R0)) #*(1-alpha_direct)

In [9]:
def calculate_diffuse_reflection_from_chl(λ, chl, alpha_chl, alpha_w, beta_w, σ, alpha_direct):
    #  In the case of ocean interior reflectance for direct incoming radiation it depends on µ = cos(θ) whereas in the
    # case of ocean interior reflectance for diffuse µ = 0.676. This value is considered an effective angle of incoming radiation of 47.47◦
    # according to Morel and Gentili (1991). Hence
    return calculate_direct_reflection_from_chl(λ, chl, alpha_chl, alpha_w, beta_w, σ, np.arccos(0.676), alpha_direct)

In [10]:
def whitecap(wind):
    # Whitecap effect as defined by Salisbury et al. 2014. NOTE that the value in paper is in percent
    # so we use the ratio instead (/100.) 
    # Salisbury, D. J., Anguelova, M. D., and Brooks, I. M.: Global Distribution and Seasonal 
    # Dependence of Satellite-based Whitecap Fraction
    #
    # Whitecaps are the surface manifestation of bubble plumes, created when 
    # surface gravity waves break and entrain air into the water column. 
    # They enhance air-sea exchange, introducing physical processes different from 
    # those operating at the bubble-free water surface. 

    return 0.000397*(np.exp(1.59*np.log(wind)))

In [11]:
def calculate_spectral_and_broadband_OSA(wind,alpha_wc,alpha_direct,alpha_diffuse,alpha_direct_chl,alpha_diffuse_chl,solar_energy):
    wc = whitecap(wind)
    OSA_direct = (alpha_direct + alpha_direct_chl) * (1-wc) + wc*alpha_wc
    OSA_diffuse = (alpha_diffuse + alpha_diffuse_chl) * (1-wc) + wc*alpha_wc
    
    # Integrate across all wavelengths 200-4000nm at 10 nm wavelength bands and then weight by the solar energy at each band.
    # The solar energy is dimensionless with sum equal to 1 and therefore already weighted.
    OSA_direct_broadband = np.sum(OSA_direct*solar_energy)
    OSA_diffuse_broadband = np.sum(OSA_diffuse*solar_energy)
    return OSA_direct_broadband

In [12]:
def calculate_OSA(µ, uv, chl, wavelengths, refractive_indexes, alpha_chl, alpha_w, beta_w, alpha_wc, solar_energy):
    
    # Solar zenith angle
   # µ = np.cos(np.radians(latitude))
    # wind is wind at 10 m height (m/s)
    σ = np.sqrt(0.003+0.00512*uv)
                
    # Vectorize the functions
    vec_calculate_direct_reflection=np.vectorize(calculate_direct_reflection)
    vec_calculate_diffuse_reflection=np.vectorize(calculate_diffuse_reflection)
    vec_calculate_direct_reflection_from_chl=np.vectorize(calculate_direct_reflection_from_chl)
    vec_calculate_diffuse_reflection_from_chl=np.vectorize(calculate_diffuse_reflection_from_chl)

    # Direct reflection
    alpha_direct = vec_calculate_direct_reflection(refractive_indexes,µ,σ)
   
    # Diffuse reflection
    alpha_diffuse = vec_calculate_diffuse_reflection(refractive_indexes,σ)

    # Reflection from chlorophyll and biological pigments
    alpha_direct_chl = vec_calculate_direct_reflection_from_chl(wavelengths, chl, alpha_chl, alpha_w, beta_w, σ, µ, alpha_direct)
   
    # Diffuse reflection interior of water from chlorophyll
    alpha_diffuse_chl = vec_calculate_diffuse_reflection_from_chl(wavelengths, chl, alpha_chl, alpha_w, beta_w, σ, alpha_direct)

    # OSA
    return calculate_spectral_and_broadband_OSA(uv,alpha_wc,alpha_direct,alpha_diffuse,alpha_direct_chl,alpha_diffuse_chl,solar_energy)

In [13]:
def setup_parameters():
    df=pd.read_csv("data/Wavelength/Fresnels_refraction.csv", header=0, sep=";", decimal=",")
    wavelengths=df["λ"].values
    refractive_indexes=df["n(λ)"].values
    alpha_chl=df["a_chl(λ)"].values
    alpha_w=df["a_w(λ)"].values
    beta_w=df["b_w(λ)"].values
    alpha_wc=df["a_wc(λ)"].values
    solar_energy=df["E(λ)"].values
    print(df.head())
    return wavelengths, refractive_indexes, alpha_chl, alpha_w, beta_w, alpha_wc, solar_energy

# Calculate solar zenith angle using pysolar
![zenith](images/solar_zenith.jpg)

In [14]:
# http://www.jgiesen.de/astro/suncalc/calculations.htm
def calculate_zenith_angle(lat, lon, month, hour_of_day):
    lon = np.where(lon > 180, lon-360, lon)
    offset=(int(lon/15.))
   
    when = datetime.datetime(2006,month,15,hour_of_day,0,0,tzinfo=datetime.timezone(datetime.timedelta(hours=offset))).astimezone(pytz.utc)
    
    altitude_deg = pysolar.solar.get_altitude_fast(lat, lon, when)
    
    if  altitude_deg <= 0:
        zenith = 90.0
        cos_teta=0.0
        surface_light=0
    else:
        zenith = 90.0-altitude_deg
    
     #  surface_light = pysolar.radiation.get_radiation_direct(when, altitude_deg)
        cos_teta = np.cos(np.radians(zenith))
    
   # print("loc ({},{}) offset {} zenith {} cos_teta {} when {}".format(lat,lon,offset,zenith,cos_teta,when))
   # np.cos(np.radians(80))
    return cos_teta

In [15]:
latitudes=[37.46,60., 82]
months=[m+5 for m in range(1)]
lon=-122
hour_of_day=19
for lat in latitudes:
    for month in months:
        cosT = calculate_zenith_angle(lat,lon,month,hour_of_day)
        print("Latitude: {} cosTeta: {} month: {}".format(lat,cosT,month+1))

Latitude: 37.46 cosTeta: 0.01760787305257004 month: 6
Latitude: 60.0 cosTeta: 0.16855708349496232 month: 6
Latitude: 82 cosTeta: 0.29121536469314574 month: 6


In [ ]:
def calculate_light(config_pices_obj):
    
    #vec_calculate_insolation_weighted_zenith_angle=np.vectorize(calculate_insolation_weighted_zenith_angle)
    
    selected_time=0
    show_plot=False
    wavelengths, refractive_indexes, alpha_chl, alpha_w, beta_w, alpha_wc, solar_energy = setup_parameters()
    startdate=datetime.datetime.now()
    
    for key in config_pices_obj.dset_dict.keys():
        
        var_name = key.split("_")[0]
        model_name = key.split("_")[3]
        
        if var_name=="uas":
            
            key_v="vas"+key[3:]
            key_tos="tos"+key[3:]
            var_name_v = key_v.split("_")[0]
            model_name_v = key_v.split("_")[3]
            
            print("=> model: {} variable name: {}".format(key, var_name))
            print("=> model: {} variable name: {}".format(key_v, var_name_v))
            
            if model_name_v==model_name:
                ds_uas=config_pices_obj.dset_dict[key].isel(time=selected_time)
               # ds_uas=ds_uas.sel(y=slice(0,90))
                ds_vas=config_pices_obj.dset_dict[key_v].isel(time=selected_time)
                ds_tos=config_pices_obj.dset_dict[key_tos].isel(time=selected_time)
              #  ds_vas=ds_vas.sel(y=slice(0,90))
                  
            
                ds_out = xe.util.grid_global(1, 1)
                regridder_v = xe.Regridder(ds_vas, ds_out, 'bilinear', reuse_weights=True)
                regridder_u = xe.Regridder(ds_uas, ds_out, 'bilinear', reuse_weights=True)
                regridder_tos = xe.Regridder(ds_tos, ds_out, 'bilinear', reuse_weights=True)
                
                dr_out_vas = regridder_v(ds_vas.vas.T)
                dr_out_uas = regridder_u(ds_uas.uas.T)
                dr_out_tos = regridder_tos(ds_tos.tos)
                
                # For a for a rectangular grid the cosine of the latitude is proportional to the grid cell area.
                area_ds_uas = dr_out_uas*np.cos(np.deg2rad(dr_out_uas.y))
                area_ds_vas = dr_out_vas*np.cos(np.deg2rad(dr_out_vas.y))
                
          #      area_ds_uas = area_ds_uas.assign_coords(x=(((area_ds_uas.x + 180) % 360) - 180)).sortby('x')
          #      area_ds_uas = area_ds_uas.assign_coords(lon=(((area_ds_uas.lon + 180) % 360) - 180))
          #      area_ds_vas = area_ds_vas.assign_coords(x=(((area_ds_vas.x + 180) % 360) - 180)).sortby('x')
          #      area_ds_vas = area_ds_vas.assign_coords(lon=(((area_ds_vas.lon + 180) % 360) - 180))
                   
              #  area_ds_vas = area_ds_vas.roll(x=-180)
               
            #    proj=ccrs.PlateCarree()
                uv=(np.sqrt(area_ds_uas**2+area_ds_vas**2))
            
                print(uv)
                wind=uv.values
                m=len(wind[:,0])
                n=len(wind[0,:])
                month=6
                print("uv",uv)
                all_zens=[]
                all_OSA=[]
                for hour_of_day in range(11,12,1):
                    print("Running for hour {}".format(hour_of_day))
                    zen = [dask.delayed(calculate_zenith_angle)(uv.lat.values[i,j],
                                 uv.lon.values[i,j],month,hour_of_day)
                                 for i in range(m) 
                                  for j in range(n)]

                    zeniths = dask.compute(zen)
                    zens = np.asarray(zeniths).reshape((m, n))
                    
                    all_zens.append(zens)
                    
                    chl=0.2

                    zr = [dask.delayed(calculate_OSA)(zens[i,j], wind[i,j], chl, wavelengths, refractive_indexes, 
                                                    alpha_chl, alpha_w, beta_w, alpha_wc, solar_energy) 
                                  for i in range(m) 
                                  for j in range(n)]

                    z = dask.compute(zr)
                    print("Median albedo for  direct light {}".format(np.median(z)))

                    OSA_direct_broadband = np.array(z).reshape((m, n))
                    print("Time to finish {} with mean OSA {}".format(datetime.datetime.now()-startdate,
                          np.mean(OSA_direct_broadband)))
                    ax3 = plt.axes(projection=proj)
                    levels=np.arange(0,0.1,0.01)
                    plt.contourf(uv.lon.values,uv.lat.values,OSA_direct_broadband, levels,
                                 transform=ccrs.PlateCarree(),
                                 cmap='nipy_spectral',zorder=0, extend="both")

                    ax3.add_feature(cfeature.LAND, facecolor="lightgrey")
                    ax3.add_feature(cfeature.COASTLINE, edgecolor="black")
                    ax3.add_feature(cfeature.BORDERS, linestyle=':')
                    plt.colorbar()
                    plt.show()
                    all_OSA.append(OSA_direct_broadband)
                    
                zes=np.asarray(all_zens)
                zens=np.mean(zes,axis=0)
                
                OSA_direct_broadband=np.mean(np.asarray(all_OSA),axis=0)
             #   OSA_direct_broadband, lon = add_cyclic_point(OSA_direct_broadband, coord=uv.lon.values)
               
                ax2 = plt.axes(projection=proj)
            
                levels=np.arange(0,0.1,0.01)
                plt.pcolormesh(uv.lon.values,uv.lat.values,zens, #levels,
                             transform=ccrs.PlateCarree(),
                             cmap='nipy_spectral',zorder=0)

                ax2.add_feature(cfeature.LAND, color="lightgrey")
                ax2.add_feature(cfeature.COASTLINE, edgecolor="black")
                ax2.add_feature(cfeature.BORDERS, linestyle=':')
                plt.colorbar()
                plt.show()
                
                print(OSA_direct_broadband)
                ax3 = plt.axes(projection=proj)
                levels=np.arange(0,0.1,0.01)
                plt.contourf(uv.lon.values,uv.lat.values,OSA_direct_broadband, levels,
                             transform=ccrs.PlateCarree(),
                             cmap='nipy_spectral',zorder=0, extend="both")

                ax3.add_feature(cfeature.LAND, facecolor="lightgrey")
                ax3.add_feature(cfeature.COASTLINE, edgecolor="black")
                ax3.add_feature(cfeature.BORDERS, linestyle=':')
                plt.colorbar()
                plt.show()


              
calculate_light(config_pices_obj)

/opt/conda/lib/python3.7/site-packages/xesmf/backend.py:36: UserWarning: Input array is not F_CONTIGUOUS. Will affect performance.
  warnings.warn("Input array is not F_CONTIGUOUS. "
/opt/conda/lib/python3.7/site-packages/xarray/core/dataarray.py:1924: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, these coordinates will be transposed as well unless you specify transpose_coords=False.
  return self.transpose()
/opt/conda/lib/python3.7/site-packages/xarray/core/dataarray.py:1924: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, these coordinates will be transposed as well unless you specify transpose_coords=False.
  return self.transpose()
distributed.protocol.core - CRITICAL - Failed to Serialize
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/distributed/protocol/core.py", line 40, in dumps
    for key, value in data.items()
  File "/opt/conda/lib/python3.7/site-packages/distrib

       λ      E(λ)  n(λ)  a_chl(λ)  a_w(λ)  b_w(λ)  a_wc(λ)
0  200.0  0.000054  1.45     0.775   3.070  0.1510      0.0
1  210.0  0.000206  1.44     0.752   1.990  0.1190      0.0
2  220.0  0.000349  1.42     0.730   1.310  0.0995      0.0
3  230.0  0.000384  1.41     0.708   0.928  0.0820      0.0
4  240.0  0.000292  1.40     0.685   0.718  0.0685      0.0
=> model: uas_ssp585_gn_CanESM5_r1i1p1f1 variable name: uas
=> model: vas_ssp585_gn_CanESM5_r1i1p1f1 variable name: vas
Reuse existing file: bilinear_128x64_180x360.nc
Reuse existing file: bilinear_128x64_180x360.nc
Reuse existing file: bilinear_291x360_180x360.nc
<xarray.DataArray (y: 180, x: 360)>
dask.array<sqrt, shape=(180, 360), dtype=float64, chunksize=(180, 360), chunktype=numpy.ndarray>
Coordinates:
    height   float64 10.0
    time     object 1950-01-16 12:00:00
    lon      (y, x) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat      (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
Dimension

In [ ]:
client.close()